In [23]:
import numpy as np
import pandas as pd
import functions.multi_occupation_network as multi_occupation_network

# Calibration Excersize 1: Single Occupation

Suppose there is just one type of labor, in other words that $\mathcal{O}=1$. We calibrate our model to mathc BEA input-output tables, and treat the residual, the part of production not accounted for by intermediate inputs, as the labor elasticity. 

In [24]:
data_dir = '../data/clean/'
dfA      = pd.read_csv(data_dir + 'A.csv')
dfParam  = pd.read_csv(data_dir + 'params.csv')
dfLshare = pd.read_csv(data_dir + 'labor_share.csv')
dfLabor_market_yearly= pd.read_csv(data_dir + 'labor_market_yearly.csv')
dfLabor_market_yearly = dfLabor_market_yearly.sort_values(by=['year', 'BEA_sector'])
dfLabor_market_yearly = dfLabor_market_yearly.dropna(axis=0)
dfLabor_market_yearly = dfLabor_market_yearly[dfLabor_market_yearly['year'] == 2005]
# reformatting parameters
Omega = np.array(dfA.iloc[:, 1:], dtype='float64')
Psi = np.linalg.inv(np.eye(Omega.shape[0])-Omega)
J = Omega.shape[0]
O = 1

epsN = np.array(dfParam.α).reshape((J,O))
epsD = np.array(dfParam.θ_alt).reshape((J,1))

# labor market stats
θ = dfLabor_market_yearly['v'].sum() / dfLabor_market_yearly['u'].sum() 
ν = np.ones(O) * 0.5
curlyQ = np.diag(-ν)
curlyF =  np.eye(O) + curlyQ
r = 0.0282
curlyT = np.diag(r / (θ**(-ν) - r))
# NOTE: here I am picking the r so the τ's roughly look right. Down the line, we can calibrate τ directly from the data. 
# Also note that tightness is computed from unemployed workers, but in our model it should be computed from the whole work force.

curlyL = np.array(dfLabor_market_yearly['e']/dfLabor_market_yearly['e'].sum()).reshape((O,J))


First, we define the shocks we are interested in to allow us to easily change these in the future, that is we define the vectors $d\log\bm{A}$ and $d\log\bm{H}$.

In [25]:
dlog_A = np.zeros((J, 1))
dlog_A[3] = 0.01 
dlog_H = np.zeros((O,1))

We assume Cobb-Douglas production and preferences.

In [26]:
dlog_lam = np.zeros_like(dlog_A)
dlog_epsN = np.zeros_like(epsN)
dlog_epsD = np.zeros_like(epsD)

Finally, we need to make some assumption about how wages change. We begin by assuming that wages net of sectoral employment weighted prices do not change in response to changes in either productivity or the labor force.

In [27]:
epsW_A = np.zeros((O,J))
epsW_H = np.zeros((O,O))
dlog_wR = multi_occupation_network.WageFunc(dlog_A, dlog_H, epsW_A, epsW_H)

With these pieces in hand, we can now estimate responses to tightness. For details, see the multiple occupations example code.

In [28]:
curlyE = multi_occupation_network.curlyEFunc(dlog_epsN,epsN)
dlog_theta = multi_occupation_network.ThetaFunc(dlog_A, dlog_H, dlog_wR, dlog_epsN, dlog_lam, Psi, Omega, curlyF, curlyQ, curlyT, curlyE, curlyL, epsN)
dlog_theta

array([[0.32114894]])

In [29]:
num = 0
dlog_p = multi_occupation_network.PriceFunc(dlog_A, dlog_wR, dlog_theta, Psi, curlyQ, epsN, curlyT, curlyL, num=num)
dlog_p

array([[ 0.00000000e+00],
       [ 1.51291126e-05],
       [-6.33827206e-03],
       [-1.95856091e-02],
       [ 2.42313346e-04],
       [ 8.66557770e-04],
       [-1.18828419e-04],
       [-8.50104220e-05],
       [-1.12048795e-03],
       [-2.78191602e-03],
       [-2.60782739e-03],
       [-9.30805219e-04],
       [ 1.69910558e-04],
       [-1.10237000e-03],
       [ 2.60390344e-04],
       [-6.19598947e-04],
       [ 3.98151742e-04]])

In [30]:
dlog_y = multi_occupation_network.OutputFunc(dlog_A,dlog_H, dlog_theta, dlog_lam, Psi, Omega, curlyQ, curlyF, epsN, curlyT, curlyE)
dlog_y

array([[0.1587749 ],
       [0.15875977],
       [0.16511317],
       [0.17836051],
       [0.15853259],
       [0.15790834],
       [0.15889373],
       [0.15885991],
       [0.15989539],
       [0.16155682],
       [0.16138273],
       [0.15970571],
       [0.15860499],
       [0.15987727],
       [0.15851451],
       [0.1593945 ],
       [0.15837675]])

We can check labor market clearing holds.

In [31]:
multi_occupation_network.LaborSupply(dlog_H, dlog_theta, curlyF) - multi_occupation_network.LaborDemand(dlog_wR, dlog_y, dlog_p, dlog_epsN, curlyL)

array([[2.77555756e-17]])

Finally, we can calculate changes in aggregate output.

In [32]:
dlog_aggY = multi_occupation_network.AggOutputFunc(dlog_y, dlog_lam, dlog_epsD, epsD)
dlog_aggY

array([[0.16405142]])

Since labor market frictions are the only source of inefficiency in our network economy, in the absence of labor market frictions our network economy is fully efficient. It is well known that Hulten's theorem holds in efficient network economies with Cobb-Douglas technology. We can therefore easily compare the implications of labor market frictions for aggregate output with the implications of the same technology shocks absent labor market frictions.  

In [33]:
epsD.T @ Psi @ dlog_A

array([[0.00717491]])

We can also compare sector level output changes with and without labor market frictions.